In [6]:
# Instalação de bibliotecas no Jupyter Notebook
!pip install requests pandas openpyxl python-dotenv


In [7]:
from dotenv import load_dotenv  # Para carregar o arquivo .env
import os  # Para acessar variáveis do sistema
import requests  # Para fazer as requisições à API
import pandas as pd  # Para manipular os dados e salvar na planilha


In [8]:
# Carregar o arquivo .env
load_dotenv()

# Obter a chave de API do Google Places
API_KEY = os.getenv("API_KEY")

# Verificar se a chave foi carregada corretamente
if not API_KEY:
    raise ValueError("A chave de API não foi encontrada. Verifique o arquivo .env.")


In [9]:
# URLs da API do Google Places
search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
details_url = "https://maps.googleapis.com/maps/api/place/details/json"

# Parâmetros da busca
params = {
    "query": "clínicas médicas no Rio de Janeiro",  # Altere para o tipo de negócio e localização desejados
    "key": API_KEY
}


In [10]:
# Fazendo a requisição de busca
response = requests.get(search_url, params=params)
data = response.json()

# Lista para armazenar os dados dos negócios
negocios = []

# Limite de resultados
limite = 10
contador = 0

if "results" in data:
    for place in data["results"]:
        if contador >= limite:
            break  # Para o loop após atingir o limite

        # Coleta informações básicas
        place_id = place.get("place_id")
        name = place.get("name", "N/A")
        address = place.get("formatted_address", "N/A")
        rating = place.get("rating", "N/A")

        # Requisição para Place Details para obter telefone e site
        details_params = {
            "place_id": place_id,
            "fields": "formatted_phone_number,website",
            "key": API_KEY
        }
        details_response = requests.get(details_url, params=details_params)
        details_data = details_response.json()

        # Coletando telefone e site
        phone = details_data.get("result", {}).get("formatted_phone_number", "N/A")
        website = details_data.get("result", {}).get("website", "N/A")

        # Adicionando os dados à lista
        negocios.append({
            "Nome": name,
            "Telefone": phone,
            "Site/Instagram": website,
            "Endereço": address,
            "Avaliação": rating
        })

        contador += 1  # Incrementa o contador

else:
    print("Nenhum resultado encontrado.")


In [13]:
print(f"Status da requisição: {response.status_code}")
print("Resposta da API:")
print(data)


Status da requisição: 200
Resposta da API:
{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}


In [11]:
# Verifica se há dados coletados
if negocios:
    # Convertendo a lista para um DataFrame do pandas
    df = pd.DataFrame(negocios)

    # Salvando os dados em uma planilha Excel
    df.to_excel("negocios_locais.xlsx", index=False)
    print(f"Planilha 'negocios_locais.xlsx' salva com {len(negocios)} resultados!")
else:
    print("Nenhum dado para salvar.")


Nenhum dado para salvar.
